**Hand Eye Calibration with Charuco Board** (opencv-python 4.5.5.62)

Data preparation

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation
import glob

In [ ]:
# Load images
images = [cv.imread(image_path) for image_path in glob.glob('images/*.png')]

# Load optical marker poses
df = pd.read_csv('tf_poses_camhand.csv', header=None, names=['timestamps', 'translation_x', 'translation_y', 'translation_z', 'quaternion_x', 'quaternion_y', 'quaternion_z', 'quaternion_w'])
translations = df[['translation_x', 'translation_y', 'translation_z']].values
quaternions = df[['quaternion_x', 'quaternion_y', 'quaternion_z', 'quaternion_w']].values

rotation_matrices = []
for quaternion in quaternions:
    # Normalize quaternion
    quaternion /= np.linalg.norm(quaternion)

    # Convert quaternion to rotation matrix
    rotation_matrix = Rotation.from_quat(quaternion).as_matrix()
    rotation_matrices.append(rotation_matrix)

Calibration

In [ ]:
# Define ChArUco board parameters
CHARUCOBOARD_COLCOUNT = 9
CHARUCOBOARD_ROWCOUNT = 11
ARUCO_DICT = cv.aruco.Dictionary_get(cv.aruco.DICT_4X4_1000)
SQUARE_LENGTH = 0.047  # Checker size in meters
MARKER_LENGTH = 0.035  # Marker size in meters

# Create ChArUco board
board = cv.aruco.CharucoBoard_create(
    squaresX=CHARUCOBOARD_COLCOUNT,
    squaresY=CHARUCOBOARD_ROWCOUNT,
    squareLength=SQUARE_LENGTH,
    markerLength=MARKER_LENGTH,
    dictionary=ARUCO_DICT
)

In [ ]:
# Detect ChArUco corners and IDs
all_corners = []
all_ids = []

# Iterate through images to collect calibration data
for image in images:
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    # Detect ArUco markers
    corners, ids, _ = cv.aruco.detectMarkers(gray, ARUCO_DICT)
   
    if len(corners) > 0:
        # Refine detected markers
        corners, ids, _, _ = cv.aruco.refineDetectedMarkers(
            image=gray,
            board=board,
            detectedCorners=corners,
            detectedIds=ids,
            rejectedCorners=None,
            cameraMatrix=None,
            distCoeffs=None
        )

        if len(corners) > 0:
            # Interpolate charuco corners
            ret, charuco_corners, charuco_ids = cv.aruco.interpolateCornersCharuco(
                markerCorners=corners,
                markerIds=ids,
                image=gray,
                board=board
            )
            
            # Check if interpolation was successful and the number of corners is sufficient
            if ret > 3:
                all_corners.append(charuco_corners)
                all_ids.append(charuco_ids)


In [ ]:
# Calibrate camera
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv.aruco.calibrateCameraCharuco(
    charucoCorners=all_corners,
    charucoIds=all_ids,
    board=board,
    imageSize=gray.shape[::-1],
    cameraMatrix=None,
    distCoeffs=None
)

print("Camera calibration successful:", ret)
print("Camera Matrix:\n", camera_matrix)
print("Distortion Coefficients:\n", dist_coeffs)

In [ ]:
# Construct image_poses
image_poses = all_corners

# Construct pattern_poses using rotation_matrices and translations
pattern_poses = np.array([np.hstack((rotation_matrices[i], translations[i][:, np.newaxis])) for i in range(len(images))])

# TODO: Why "orientation" of camera, and "position" of robot arm
A = np.zeros((3 * (len(pattern_poses)-1), 3)) # changes in camera's orientation
B = np.zeros((3 * (len(pattern_poses)-1), 1)) #  changes in robot arm's position

for i, (T1, T2) in enumerate(zip(pattern_poses[:-1], pattern_poses[1:])):
    # Calculate equations for matrix A representing changes in camera's coordinate system
    # by subtracting rotation component of T1 from identity matrix
    A[3*i:3*(i+1)] = np.eye(3) - T1[:3, :3]

    # Calculate equations for matrix B representing changes in robot arm's coordinate system
    # by subtracting translation component of T1 from translation component of T2
    B[3*i:3*(i+1)] = T2[:3, 3:4] - T1[:3, 3:4]

calibration_result = np.linalg.lstsq(A, B, rcond=None)[0]

print("Calibration Result:", calibration_result)

Reprojection

In [ ]:
# TODO